<a href="https://colab.research.google.com/github/Jakkai1001/Chipotle-Visulaization/blob/main/Exercise%201%20Chipotle%20Visulaization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import plotly.express as px

url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
chipotle = pd.read_csv(url, sep='\t')

chipotle['item_price'] = chipotle['item_price'].str.replace('$', '').astype(float)

sales_by_item = chipotle.groupby('item_name').agg({'quantity': 'sum', 'item_price': 'mean'})
sales_by_item['total_sales'] = sales_by_item['quantity'] * sales_by_item['item_price']
sales_by_item = sales_by_item.sort_values('total_sales', ascending=False).reset_index()

# 1. กราฟ Bar chart: เมนูที่ขายดีที่สุด (ยอดขายรวมตามจำนวนที่ขาย)
print('''คำถาม 1: เมนูไหนที่ขายดีที่สุด?''')
print('''คำตอบ 1: จากกราฟ เมนูที่ขายดีที่สุดคือเมนูที่มีจำนวนขายสูงสุด (เช่น 'Chicken Bowl')''')

fig1 = px.bar(sales_by_item.head(10), x='item_name', y='quantity',
              title='Top 10 เมนูที่ขายดีที่สุด (จากจำนวนที่ขาย)',
              labels={'quantity': 'จำนวนที่ขาย', 'item_name': 'เมนูอาหาร'})
fig1.show()

คำถาม 1: เมนูไหนที่ขายดีที่สุด?
คำตอบ 1: จากกราฟ เมนูที่ขายดีที่สุดคือเมนูที่มีจำนวนขายสูงสุด (เช่น 'Chicken Bowl')


In [13]:
# 2. กราฟ Bar chart: เมนูที่มีราคาสูงสุด (เฉลี่ยราคา)
print('''คำถาม 2: เมนูไหนที่มีราคาสูงสุด?  ''')
print('''คำตอบ 2: จากกราฟ เมนูที่มีราคาสูงสุดคือเมนูที่มีราคาเฉลี่ยต่อชิ้นสูงที่สุด (เช่น 'Bowl')''')
fig2 = px.bar(sales_by_item.sort_values('item_price', ascending=False).head(10),
              x='item_name', y='item_price',
              title='Top 10 เมนูที่มีราคาสูงสุด (เฉลี่ยราคา)',
              labels={'item_price': 'ราคาเฉลี่ย ($)', 'item_name': 'เมนูอาหาร'})
fig2.show()

คำถาม 2: เมนูไหนที่มีราคาสูงสุด?  
คำตอบ 2: จากกราฟ เมนูที่มีราคาสูงสุดคือเมนูที่มีราคาเฉลี่ยต่อชิ้นสูงที่สุด (เช่น 'Bowl')


In [14]:
chipotle['total_sales'] = chipotle['quantity'] * chipotle['item_price']
sales_by_item = chipotle.groupby('item_name')['total_sales'].sum().reset_index()

sales_by_item = sales_by_item.sort_values(by='total_sales', ascending=False).reset_index(drop=True)

sales_by_item['cumulative_sales'] = sales_by_item['total_sales'].cumsum()
total_sales_sum = sales_by_item['total_sales'].sum()
sales_by_item['cumulative_percent'] = 100 * sales_by_item['cumulative_sales'] / total_sales_sum

fig = go.Figure()

fig.add_trace(go.Bar(
    x=sales_by_item['item_name'],
    y=sales_by_item['total_sales'],
    name='ยอดขายรวมต่อสินค้า',
    marker_color='steelblue',
))

fig.add_trace(go.Scatter(
    x=sales_by_item['item_name'],
    y=sales_by_item['cumulative_percent'],
    name='เปอร์เซ็นต์สะสม',
    yaxis='y2',
    mode='lines+markers',
    marker_color='orange',
))

fig.update_layout(
    title='Pareto Chart: ยอดขายรวมต่อสินค้า Chipotle',
    xaxis=dict(title='สินค้า (item_name)', tickangle=-45),
    yaxis=dict(title='ยอดขายรวม ($)'),
    yaxis2=dict(
        title='เปอร์เซ็นต์สะสม (%)',
        overlaying='y',
        side='right',
        range=[0, 110],
        showgrid=False,
    ),
    legend=dict(x=0.75, y=1.15, orientation='h'),
    margin=dict(b=150),
    height=600,
)

fig.show()